In [1]:
from datetime import datetime
datetime.strptime('15/06/2022', "%d/%m/%Y").strftime("%y-%M-%D")

'22-00-06/15/22'

In [3]:
from base64 import b64encode

# Encode the username and password for Basic Authentication
credentials = b64encode("admin:bru".encode()).decode("utf-8")
headers = {
    "Authorization": f"Basic {credentials}"
}
headers

{'Authorization': 'Basic YWRtaW46YWRtaW4='}

In [4]:
import chromadb
from chromadb.config import Settings
from decouple import config
chroma_client = chromadb.HttpClient(settings=Settings(allow_reset=True,
 ), host = "localhost", port = 8001, headers=headers)
# chroma_client.reset()


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
chromadb = Chroma(client = chroma_client, collection_name = "smart_maya", embedding_function=OpenAIEmbeddings())

In [6]:
chromadb.similarity_search("Impot")

[Document(page_content='33337616255361 18\nV17.02.06.01.10020115 3761625536 ACO FRFR\nNuméro de l\'avis\nde contravention\n3761625536\nLiberté Égalité Fraternité\nRÉPUBLIQUE FRANÇAISE\nAVIS DE CONTRAVENTION\nwww.antal.gouv.fr est l\'unique site officiel habilité vous permettant de\nréaliser gratuitement toutes vos démarches en ligne dont les\ncontestations.\nMadame, Monsieur,\nLe véhicule dont le certificat d\'immatriculation est établi\nà votre nom a fait l\'objet d\'un contrôle ayant permis de\nconstater l\'infraction figurant ci-dessous.\nT5 v17 02\nDESCRIPTION DE L\'INFRACTION\nEXCES DE VITESSE\nEXCES DE VITESSE INFERIEUR A 20 KM/H PAR CONDUCTEUR DE VEHICULE A MOTEUR -\nVITESSE MAXIMALE AUTORISEE INFERIEURE OU EGALE A 50 KM/H\n- Prévue par Art. R. 413-14 §1 du C. de la route.\nRéprimée par Art. R. 413-14 §I al. 1 du C. de la route.\nDate/heure de constatation : le 15/06/2023 à 10h08\n75 av de la division leclerc\n.PK/PR: 000.000\nDirection sarcelles VERS montmorency\nSt brice sous 

In [ ]:
# create the chroma client
import chromadb
import uuid
from chromadb.config import Settings

client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("my_collection")
for doc in docs:
    collection.add(
        ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
    )

# tell LangChain to use our client and collection name
db4 = Chroma(client=client, collection_name="my_collection", embedding_function=embedding_function)
query = "What did the president say about Ketanji Brown Jackson"
docs = db4.similarity_search(query)
print(docs[0].page_content)

In [17]:
import chromadb
import uuid
from chromadb.config import Settings

In [19]:
client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset() 

Exception: {"detail":"Not Found"}

In [ ]:
from decouple import config
from base64 import b64encode
logs = "admin:{}".format(config("ADMIN"))
logs_byes = logs.encode("utf-8")
encoded_logs = b64encode(logs_byes)
encoded_logs.decode("utf-8")